<a href="https://colab.research.google.com/github/alonba/eeg-bss/blob/main/FastICA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne
!pip install pymatreader

In [2]:
import numpy as np
import mne as mne

In [ ]:
# Fetch the data from the .set file
eeglab_epochs = mne.io.read_epochs_eeglab('./35_electrodes_filtered_epoched.set')

# Create the train set and test set
total_num_of_epochs = len(eeglab_epochs)
train_set_size = int(total_num_of_epochs*0.8)
test_set_size = total_num_of_epochs - train_set_size

train_set_indexes = []
test_set_indexes = []
for i in range(total_num_of_epochs):
    random_num = np.random.rand()
    if (random_num < 0.8):  # 80% chance of being here
        train_set_indexes.append(i)
    else:
        test_set_indexes.append(i)

train_set = eeglab_epochs[train_set_indexes]
test_set = eeglab_epochs[test_set_indexes]

# Export the Data
mne.export.export_epochs('train_set.set',train_set)
mne.export.export_epochs('test_set.set',test_set)

In [3]:
train_set = mne.io.read_epochs_eeglab('train_set.set')
train_ica = mne.preprocessing.ICA(n_components=35)  # Defaults to FastICA
train_ica.fit(train_set)

Extracting parameters from c:\Users\alonb\OneDrive - Technion\טכניון\סמסטר ט' - אביב 2022\פרוייקט א\eeg-bss\train_set.set...
Not setting metadata
380 matching events found
No baseline correction applied
0 projection items activated
Ready.
Fitting ICA to data using 35 channels (please be patient, this may take a while)


C:\Users\alonb\AppData\Local\Temp\ipykernel_9860\2758163773.py:3: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  train_ica.fit(train_set)


Selecting by number: 35 components


KeyboardInterrupt: 

In [12]:
train_set_S1=train_set['S  1']
train_set_S2=train_set['S  2']
train_set_S4=train_set['S  4']

In [13]:
train_S1_reconstructed = []
train_S2_reconstructed = []
train_S4_reconstructed = []
for i in range(35):
    tmp_train_S1 = train_set_S1.copy()
    tmp_train_S2 = train_set_S2.copy()
    tmp_train_S4 = train_set_S4.copy()
    train_S1_reconstructed.append(train_ica.apply(tmp_train_S1,include=i))
    train_S2_reconstructed.append(train_ica.apply(tmp_train_S2,include=i))
    train_S4_reconstructed.append(train_ica.apply(tmp_train_S4,include=i))


Applying ICA to Epochs instance
    Transforming to ICA space (35 components)
    Zeroing out 34 ICA components
    Projecting back using 35 PCA components
Applying ICA to Epochs instance
    Transforming to ICA space (35 components)
    Zeroing out 34 ICA components
    Projecting back using 35 PCA components
Applying ICA to Epochs instance
    Transforming to ICA space (35 components)
    Zeroing out 34 ICA components
    Projecting back using 35 PCA components
Applying ICA to Epochs instance
    Transforming to ICA space (35 components)
    Zeroing out 34 ICA components
    Projecting back using 35 PCA components
Applying ICA to Epochs instance
    Transforming to ICA space (35 components)
    Zeroing out 34 ICA components
    Projecting back using 35 PCA components
Applying ICA to Epochs instance
    Transforming to ICA space (35 components)
    Zeroing out 34 ICA components
    Projecting back using 35 PCA components
Applying ICA to Epochs instance
    Transforming to ICA space (3